In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from src.data_processing import load_data
import itertools
import string

from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag
from src.data_processing import load_cmu
from src.ipatoarpabet import translate
from string import punctuation
from src.pronunciations import phonetic_distance
import os
from pattern.en import lexeme
from src.pronunciations import get_closest_sounding_words as csw
from src.pun_algorithms import is_Tom_Swifty
from gensim import models

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [60]:
task1, task2, task3, min_pairs, strings, pun_strings = load_data()

In [15]:
model = models.KeyedVectors.load_word2vec_format('/home/doogy/Data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [26]:
def classification_accuracy(run, labels):
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for tag, label in zip(run, labels):
        if tag and label:
            tp += 1
        if not tag and not label:
            tn += 1
        if tag and not label:
            fp += 1
        if not tag and label:
            fn += 1
    print(len(labels))
    results = {}
    results['acc'] = (tp + tn) / len(labels)
    results['recall'] = tp / (tp + fn)
    results['prec'] = tp / (tp + fp)
    results['f1'] = ((2*(results['recall'] * results['prec']))
                      / (results['recall'] + results['prec']))
    
    return results

In [7]:
tom_swifties = [context for context in task1 if 'Tom' in context['words'] or not context['pun']]
run = []
for i, t in enumerate(tom_swifties):
    run.append(bool(is_Tom_Swifty(' '.join(t['words']), model)))
    print_progress(i, len(tom_swifties))

checking
 |----------------------------------------------------------------------------------------------------| 0.0% checking
 |----------------------------------------------------------------------------------------------------| 0.4% checking
 |█---------------------------------------------------------------------------------------------------| 0.7% checking
 |█---------------------------------------------------------------------------------------------------| 1.1% checking
 |██--------------------------------------------------------------------------------------------------| 1.9% checking
 |██--------------------------------------------------------------------------------------------------| 2.0% checking
 |███-------------------------------------------------------------------------------------------------| 3.4% checking
 |████------------------------------------------------------------------------------------------------| 3.7% checking
 |████-----------------------------------------

 |███████████████████████████████████████████████████████████████████---------------------------------| 67.0% checking
 |████████████████████████████████████████████████████████████████████--------------------------------| 67.6% checking
 |████████████████████████████████████████████████████████████████████--------------------------------| 67.7% checking
 |████████████████████████████████████████████████████████████████████--------------------------------| 68.3% checking
checking
 |██████████████████████████████████████████████████████████████████████------------------------------| 70.0% checking
 |███████████████████████████████████████████████████████████████████████-----------------------------| 70.6% checking
 |███████████████████████████████████████████████████████████████████████-----------------------------| 71.0% checking
 |███████████████████████████████████████████████████████████████████████-----------------------------| 71.1% checking
 |█████████████████████████████████████

In [13]:
classification_accuracy(run, [c['pun'] for c in tom_swifties])

{'acc': 0.9632152588555858,
 'f1': 0.9370629370629371,
 'prec': 0.9852941176470589,
 'recall': 0.8933333333333333}

In [3]:
jts = {'acc': 0.9632152588555858,
 'f1': 0.9370629370629371,
 'prec': 0.9852941176470589,
 'recall': 0.8933333333333333}

In [8]:
def row_to_string(row_name, d):
    return (row_name
            + ", {0:.3f}".format(d['acc'])
            + ", {0:.3f}".format(d['prec'])
            + ", {0:.3f}".format(d['recall'])
            + ", {0:.3f}\n".format(d['f1']))

In [15]:
run = []
for i, t in enumerate(task1):
    run.append(bool(is_Tom_Swifty(' '.join(t['words']), model)))
    print_progress(i, len(task1))

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242.4% 

In [29]:
classification_accuracy(run, [bool('Tom' in c['words']) for c in task1])

1780


{'acc': 0.9792134831460674,
 'f1': 0.9157175398633257,
 'prec': 0.9392523364485982,
 'recall': 0.8933333333333333}

In [4]:
all_puns = {'acc': 0.9792134831460674,
 'f1': 0.9157175398633257,
 'prec': 0.9392523364485982,
 'recall': 0.8933333333333333}

In [9]:
with open("results/tables/tom_swifties.csv", 'w') as f:
    f.write("Dataset, Accuracy, Precision, Recall, F1" + "\n")
    f.write(row_to_string("Only Tom Swifties", jts))
    f.write(row_to_string("Entire Dataset", all_puns))

In [9]:
is_Tom_Swifty("I will never give up my hounds ! Tom said doggedly .", model)

(('dog', 'hounds'), 0.1954344153404236)

In [8]:
model.similarity("sequelae", "salivary")

0.32301864

In [12]:
detect("OLD CHINESE COOKS never die , they just wok away")

['OLD', 'CHINESE', 'COOKS', 'never', 'die', ',', 'they', 'just', 'wok', 'away']
CHINESE
OLD CHINESE COOKS never die , they just wok away
CH/AY/N/IY/Z
COOKS
OLD CHINESE COOKS never die , they just wok away
K/UH/K/S
never
OLD CHINESE COOKS never die , they just wok away
N/EH/V/ER
just
OLD CHINESE COOKS never die , they just wok away
JH/AH/S/T
wok
OLD CHINESE COOKS never die , they just wok away
W/AA/K


[('wok',
  [('hock', 1, 0),
   ('watt', 1, 0),
   ('was', 1, 0),
   ('caulk', 1, 0),
   ('sock', 1, 54),
   ('locke', 1, 0),
   ('wash', 1, 738),
   ('mock', 1, 0),
   ('wad', 1, 0),
   ('cock', 1, 0),
   ('shock', 1, 0),
   ('chock', 1, 0),
   ('dock', 1, 0),
   ('weak', 1, 0),
   ('work', 1, 246),
   ('knock', 1, 0),
   ('mach', 1, 0),
   ('bach', 1, 0),
   ('walk', 1, 47351),
   ('lock', 1, 51),
   ('woke', 1, 0),
   ('pock', 1, 0),
   ('rock', 1, 0),
   ('whack', 1, 76),
   ('jacques', 1, 0),
   ('watch', 1, 0),
   ('wick', 1, 0),
   ('chalk', 1, 0),
   ('week', 1, 0),
   ('wake', 1, 0),
   ('doc', 1, 0),
   ('wan', 1, 0),
   ('jock', 1, 0),
   ('woks', 1, 0),
   ('juan', 1, 0)]),
 ('never',
  [('nether', 1, 0), ('lever', 1, 0), ('sever', 1, 0), ('ever', 1, 0)]),
 ('just',
  [('bussed', 1, 0),
   ('joist', 1, 0),
   ('cussed', 1, 0),
   ('gust', 1, 0),
   ('joust', 1, 0),
   ('rust', 1, 0),
   ('juster', 1, 0),
   ('must', 1, 0),
   ('bused', 1, 0),
   ('dust', 1, 0),
   ('jut', 1,

In [11]:
from src.pun_algorithms import detect, translate

In [30]:
is_Tom_Swifty(' '.join(test), model)

False

In [29]:
test[-2] = "said"

In [27]:
phonetic_distance('decided', 'descents')

0.4

#### Use first one with the mountains, change words to claim and decided - new puns!

In [33]:
test = task1[1]['words']

In [70]:
test[-2] = 'needlessly'

In [71]:
is_Tom_Swifty(' '.join(test), model)

[('needlessly',
  [(('needed', 'would'), 0.2108883112668991),
   (('needler', 'like'), 0.15938220545649529),
   (('needless', 'like'), 0.15065414255315607),
   (('needless', 'like'), 0.15065414255315607),
   (('needa', 'like'), 0.14301756024360657),
   (('needn', 'like'), 0.13829976320266724),
   (('neded', 'would'), 0.13548946380615234),
   (('need', 'would'), 0.13455613092942673),
   (('need', 'would'), 0.13455613092942673),
   (('needin', 'like'), 0.12854859977960587),
   (('needful', 'like'), 0.12390783056616783),
   (('needled', 'said'), 0.12348190881311893),
   (('neeed', 'would'), 0.12339918179945512),
   (('neeed', 'would'), 0.12339918179945512),
   (('needs', 'would'), 0.11948306994004683),
   (('needing', 'would'), 0.10347170010209084),
   (('needy', 'laborer'), 0.08808032423257828),
   (('neede', 'like'), 0.08539349924434315),
   (('neede', 'like'), 0.08539349924434315),
   (('needlelike', 'like'), 0.08519475162029266),
   (('needle', 'would'), 0.08394812233746052),
   (('ne

In [76]:
toms[0][0][1][0][0]

['ledge', 'mountain']

In [49]:
task1[1]

{'pun': True,
 'words': ['I',
  'would',
  'like',
  'to',
  'be',
  'a',
  'Chinese',
  'laborer',
  ',',
  'said',
  'Tom',
  'cooly',
  '.']}

In [53]:
with open("results/tom_swifties.json") as f:
    toms = json.load(f)

In [55]:
from nltk import pos_tag

In [56]:
pos_tag(task1[1]['words'])

[('I', 'PRP'),
 ('would', 'MD'),
 ('like', 'VB'),
 ('to', 'TO'),
 ('be', 'VB'),
 ('a', 'DT'),
 ('Chinese', 'JJ'),
 ('laborer', 'NN'),
 (',', ','),
 ('said', 'VBD'),
 ('Tom', 'NNP'),
 ('cooly', 'NN'),
 ('.', '.')]

In [62]:
test[-2]="calmly"

In [63]:
test

['I',
 'would',
 'like',
 'to',
 'be',
 'a',
 'Chinese',
 'laborer',
 ',',
 'said',
 'Tom',
 'calmly',
 '.']

In [61]:
task1[1]

{'pun': True,
 'words': ['I',
  'would',
  'like',
  'to',
  'be',
  'a',
  'Chinese',
  'laborer',
  ',',
  'said',
  'Tom',
  'coolly',
  '.']}